In [ ]:
import os
import torch
import numpy as np
from PIL import Image
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info

# ПАТЧИНГ БИБЛИОТЕКИ

try:
    import tam
    from tam import TAM
except ImportError:
    print("tam.py не найден.")
    exit(1)

_original_id2idx = tam.id2idx

def patched_id2idx(inp_id, target_id, return_last=False):
    if isinstance(target_id, tuple) and target_id[0] == 'force_index':
        return target_id[1]
    return _original_id2idx(inp_id, target_id, return_last)

tam.id2idx = patched_id2idx
print("Библиотека TAM успешно пропатчена для работы с Qwen2-VL.")

MODEL_PATH = "Qwen/Qwen2-VL-2B-Instruct"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
IMAGE_PATH = "demo_image.jpg"
PROMPT_TEXT = "Describe this image in detail."
SAVE_DIR = "results_hw5"
os.makedirs(SAVE_DIR, exist_ok=True)

print(f"Загрузка модели {MODEL_PATH}...")
model = Qwen2VLForConditionalGeneration.from_pretrained(
    MODEL_PATH, torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32, device_map="auto"
)
processor = AutoProcessor.from_pretrained(MODEL_PATH)

if not os.path.exists(IMAGE_PATH):
    import requests
    url = "https://raw.githubusercontent.com/QwenLM/Qwen-VL/master/assets/demo.jpeg"
    Image.open(requests.get(url, stream=True).raw).save(IMAGE_PATH)

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": IMAGE_PATH},
            {"type": "text", "text": PROMPT_TEXT},
        ],
    }
]

text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
).to(DEVICE)

print("Генерация ответа...")
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=40,
        use_cache=True,
        output_hidden_states=True,
        return_dict_in_generate=True
    )

generated_ids = outputs.sequences[0]
input_len = inputs.input_ids.shape[1]
new_tokens = generated_ids[input_len:]

print(f"Ответ: {processor.decode(new_tokens, skip_special_tokens=True)}")

logits_list = []
logits_list.append(model.lm_head(outputs.hidden_states[0][-1]))
for state in outputs.hidden_states[1:]:
    logits_list.append(model.lm_head(state[-1]))

if 'image_grid_thw' in inputs:
    thw = inputs['image_grid_thw'][0]
    vision_shape = (int(thw[1] // 2), int(thw[2] // 2))
    print(f"Calculated Vision Shape: {vision_shape} (tokens), Original grid: {thw[1]}x{thw[2]}")
else:
    vision_shape = (24, 24)

vis_start_id = processor.tokenizer.convert_tokens_to_ids("<|vision_start|>")
vis_end_id = processor.tokenizer.convert_tokens_to_ids("<|vision_end|>")

special_ids = {
    'img_id': [vis_start_id, vis_end_id], # Тут оставляем поиск по токенам, это работает
    'prompt_id': [('force_index', 0), ('force_index', input_len - 1)],
    'answer_id': [('force_index', input_len), ('force_index', len(generated_ids) - 1)]
}

vis_inputs = [Image.open(IMAGE_PATH).convert("RGB")]
raw_map_records = []

print(f"Запуск визуализации в папку {SAVE_DIR}...")

for i, token_id in enumerate(new_tokens):
    token_str = processor.decode([token_id])
    safe_name = "".join([c if c.isalnum() else "_" for c in token_str]).strip()
    if not safe_name: safe_name = "sym"

    filename = os.path.join(SAVE_DIR, f"token_{i:02d}_{safe_name}.jpg")
    print(f"[{i}] {token_str} -> {filename}")

    try:
        TAM(
            generated_ids.cpu().tolist(),
            vision_shape,
            logits_list,
            special_ids,
            vis_inputs,
            processor,
            filename,
            i,
            raw_map_records,
            False
        )
    except Exception as e:
        print(f"  ERROR on token {i}: {e}")
        continue

print("Завершено.")

In [21]:
print("=" * 60)
print("ОТЧЕТ ПО ДЗ 5")
print("=" * 60)
print("\nРЕЗУЛЬТАТ ЭКСПЕРИМЕНТА:")
print(" Показал неудовлетворительные результаты.")

print("\nОСНОВНЫЕ ПРОБЛЕМЫ:")
print("1. [Качество визуализации]: Тепловые карты (activation maps) сильно зашумлены.")
print("   Области внимания (красные пятна) не совпадают с генерируемыми объектами")
print("   (например, подсветка фона вместо объекта или хаотичные пятна).")

print("2. [Техническая реализация]: Метод TAM оказался несовместим 'из коробки'")
print("   с архитектурой Qwen2-VL. Модель использует динамическое разрешение")
print("   и объединение патчей (patch merging), из-за чего возникают критические")
print("   ошибки размерности тензоров (Shape Mismatch), требующие ручного хардкодинга.")

print("\nВЫВОД:")
print("Применение метода TAM не выдало ожидаемого ответа")
print("=" * 60)

ОТЧЕТ ПО ДЗ 5

РЕЗУЛЬТАТ ЭКСПЕРИМЕНТА:
 Показал неудовлетворительные результаты.

ОСНОВНЫЕ ПРОБЛЕМЫ:
1. [Качество визуализации]: Тепловые карты (activation maps) сильно зашумлены.
   Области внимания (красные пятна) не совпадают с генерируемыми объектами
   (например, подсветка фона вместо объекта или хаотичные пятна).
2. [Техническая реализация]: Метод TAM оказался несовместим 'из коробки'
   с архитектурой Qwen2-VL. Модель использует динамическое разрешение
   и объединение патчей (patch merging), из-за чего возникают критические
   ошибки размерности тензоров (Shape Mismatch), требующие ручного хардкодинга.

ВЫВОД:
Применение метода TAM не выдало ожидаемого ответа
